In [ ]:
from pymongo import MongoClient
import utils.const as const

In [ ]:
client = MongoClient()
client.list_database_names() # ['admin', 'local']

In [ ]:
db=const.grpId
collection=const.PostCollection

data='IdMergedScores.csv'

mergeData='mergeData.txt'
mergeId='mergeId.txt'

In [ ]:
myCollect=client[db][collection]

In [ ]:
with open(data,'r') as fp:
    fp.readline() ## header
    for l in fp:
        arr=l.split(',')
        if len(arr)<3:
            continue
        postId=arr[0]
        commentId=arr[1]
        
        result=myCollect.update_one({'_id': postId, 'comments': {'$elemMatch': {'_id': commentId}}}, {
            '$set': {'comments.$.sentimentVal':  float(arr[2][:-1])}})
        
        if result.matched_count==0 :
            print("=="+postId+"==")
            print("=="+commentId+"==")
            print(result.matched_count , result.modified_count)
            print()

In [ ]:
for post in list(myCollect.find()):
    if 'comments' not in post or len(post['comments'])==0:
        continue
    
    for comment in post['comments']:
        if 'like_count' not in comment:
            myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
            '$set': {'comments.$.like_count':  0}})

In [ ]:

for post in list(myCollect.find()):
    if 'comments' not in post or len(post['comments'])==0:
        continue
    num=0
    denom=len(post['comments'])
    
    for comment in post['comments']:
        num+=comment['sentimentVal']*(comment['like_count']+1)
        denom+=comment['like_count']
    
    myCollect.update_one({'_id': post['_id']}, {
            "$set": {'comment_sentiment' : num/denom}
        })

In [ ]:
fp1=open(mergeId,'r')
lines=fp1.readlines()
fp1.close()

with open(mergeData,'r') as fp:
    i=0
    for l in fp:
        arr=lines[i].split(' ')
        i+=1
        if len(arr)!=2:
            continue
                
        postId=arr[0]
        commentId=arr[1][:-1]
        
        result=myCollect.update_one({'_id': postId, 'comments': {'$elemMatch': {'_id': commentId}}}, {
            '$set': {'comments.$.message':  l[:-1]}})
        
        if result.matched_count==0 :
            print("=="+postId+"==")
            print("=="+commentId+"==")
            print(result.matched_count , result.modified_count)
            print()
        